In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import glob
import os

factors = ['diff_orb', 'diff_tov', 'diff_3p%', 'diff_2p%', 'diff_ft%', 'diff_fta', 'diff_3pa']

rank_combined = pd.DataFrame()
for year in list(range(2016, 2025)):
    rank = pd.read_csv(f'/Users/moneysniper/Documents/NBA_analysis_project/standings/standing_{year}.csv', skiprows = 1)
    rank.replace({'Team':{'Golden State Warriors':'GSW', 'San Antonio Spurs':'SAS', 'Cleveland Cavaliers':'CLE', 'Toronto Raptors':'TOR', 'Oklahoma City Thunder':'OKC', 'Los Angeles Clippers':'LAC', 'Atlanta Hawks':'ATL', 'Boston Celtics':'BOS', 'Charlotte Hornets':'CHO', 'Miami Heat':'MIA', 'Indiana Pacers':'IND', 'Detroit Pistons':'DET', 'Portland Trail Blazers':'POR', 'Chicago Bulls':'CHI', 'Dallas Mavericks':'DAL', 'Washington Wizards':'WAS', 'Utah Jazz':'UTA', 'Houston Rockets':'HOU', 'Memphis Grizzlies':'MEM', 'Sacramento Kings':'SAC', 'Orlando Magic':'ORL', 'Milwaukee Bucks':'MIL', 'New York Knicks':'NYK', 'New Orleans Pelicans':'NOP', 'Minnesota Timberwolves':'MIN', 'Phoenix Suns':'PHO', 'Philadelphia 76ers':'PHI', 'Los Angeles Lakers':'LAL', 'Brooklyn Nets':'BRK', 'Denver Nuggets':'DEN'}}, inplace = True)
    rank['Season'] = year
    rank_combined = pd.concat([rank_combined, rank])

game_logs = pd.read_csv(f'/Users/moneysniper/Documents/NBA_analysis_project/gamelogs/regular_season.csv')
factors = ['diff_orb', 'diff_tov', 'diff_3p%', 'diff_2p%', 'diff_ft%', 'diff_fta', 'diff_3pa']

merged = game_logs.merge(rank_combined, left_on=['team', 'season'] , right_on=['Team', 'Season'])
merged = merged.merge(rank_combined, left_on=['team_opp', 'season'], right_on=['Team', 'Season'], suffixes=('', '_opp'))
merged['group'] = pd.cut(merged['Rk'], bins=[0, 10, 20, 30], labels=['top', 'mid', 'bottom'])
merged['opponent_group'] = pd.cut(merged['Rk_opp'], bins=[0, 10, 20, 30], labels=['top', 'mid', 'bottom'])
top_vs_top = merged[(merged['group'] == 'top') & (merged['opponent_group'] == 'top')]
mid_vs_mid = merged[(merged['group'] == 'mid') & (merged['opponent_group'] == 'mid')]
bottom_vs_bottom = merged[(merged['group'] == 'bottom') & (merged['opponent_group'] == 'bottom')]
top_vs_mid = merged[(merged['group'] == 'top') & (merged['opponent_group'] == 'mid')]
mid_vs_bottom = merged[(merged['group'] == 'mid') & (merged['opponent_group'] == 'bottom')]
top_vs_bottom = merged[(merged['group'] == 'top') & (merged['opponent_group'] == 'bottom')]

groupings = [('top_vs_top', top_vs_top), ('mid_vs_mid', mid_vs_mid), ('bottom_vs_bottom', bottom_vs_bottom), 
            ('top_vs_mid', top_vs_mid), ('mid_vs_bottom', mid_vs_bottom), ('top_vs_bottom', top_vs_bottom)]

exp_params = {}
for name, group in groupings:
        df = group.dropna(subset = factors + ['won'])
        
        X = df[factors]
        y = df['won']

        X = sm.add_constant(X)
        model = sm.Logit(y, X)
        result = model.fit()
        
        exp_params[name] = np.exp(result.params)

exp_params_df = pd.DataFrame(exp_params)
exp_params_df = exp_params_df.drop('const', axis=0)
exp_params_df.to_csv('Heterogenous.csv')  

Optimization terminated successfully.
         Current function value: 0.162681
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.154646
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.166666
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.155920
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.158272
         Iterations 9
Optimization terminated successfully.
         Current function value: 0.129805
         Iterations 9
